# [Advent of Code 2022 Day 13](https://adventofcode.com/2022/day/13)
We love a total ordering!

## Initial setup

In [1]:
from __future__ import annotations
import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2
run_doctest_for = lambda func: doctest.run_docstring_examples(func, globals())

## Test Cases

### Part 1

In [2]:
PART_ONE_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 13,
    },
    "input": {
        "input1": 5684,
    },
}
PART_ONE_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}
PART_ONE_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}

### Part 2

In [3]:
PART_TWO_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 140,
    },
    "input": {
        "input1": 22932,
    },
}
PART_TWO_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}
PART_TWO_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}

## Input Parsing
I'm a dirty little `json` abuser yes.

In [4]:
class Model(BaseModel):
    data: Any

def parse_input_from_filename(filename: str) -> Context:
    lines = list(yield_line(filename))

    ctx = Context()
    ctx.input = []

    input_lines = ctx.input

    lmao = []

    for idx, line in enumerate(lines):
        if line == "":
            input_lines.append(lmao[:])
            lmao.clear()
            continue
        lmao.append(json.loads(line))

    input_lines.append(lmao)

    return ctx

### Test Parsing Examples

In [5]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["example"].keys() | PART_TWO_CASES["example"].keys())
def test_parsing_examples(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

.                                                                                            [100%]
============================================= PASSES ==============================================
_________________________________ test_parsing_examples[example1] _________________________________
-------------------------------------- Captured stderr call ---------------------------------------
[[1, 1, 3, 1, 1], [1, 1, 5, 1, 1]]
[[[1], [2, 3, 4]], [[1], 4]]
[[9], [[8, 7, 6]]]
[[[4, 4], 4, 4], [[4, 4], 4, 4, 4]]
[[7, 7, 7, 7], [7, 7, 7]]
[[], [3]]
[[[[]]], [[]]]
[[1, [2, [3, [4, [5, 6, 7]]]], 8, 9], [1, [2, [3, [4, [5, 6, 0]]]], 8, 9]]
1 passed in 0.02s


### Test Parsing Inputs

In [6]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["input"].keys() | PART_TWO_CASES["input"].keys())
def test_parsing_inputs(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

.                                                                                            [100%]
============================================= PASSES ==============================================
___________________________________ test_parsing_inputs[input1] ___________________________________
-------------------------------------- Captured stderr call ---------------------------------------
[[[[10, [], 5, [5, 10, 2, 10, 5]]], [[[4, 1, 2, 5], 1, 2, 9], [4, 10], [], []], [[], 8, [[7], 7]], [[[8], 7, 8, 3]], [7, 5]], [[5, [[9, 4], 9], [7, [6, 4], [1, 9, 9, 3, 10]], 2, 2], [[3, [], 1, 9]], [[8, [5], 2, [10, 3, 8, 8]], 4, [], 10, [5, 9]], [[[], 4, [7, 8, 6]], [[2, 2, 1, 8, 5], [0, 10, 6, 6]], [7, [7, 9, 9, 7], 8, 6]]]]
[[[[10, 10], 9, 8, 6], [[9, 8, [2, 1, 10, 0], [6, 8]], 3, 7, [[10, 5, 10], [3, 1, 6, 5, 5]]], [[[4, 9, 5, 4, 9], 0, 7, [5], 5], 1]], [[5, 1, 2, 4, 1], [10, 4, [[3, 9], 5, 6, [5, 8, 6, 2]], 5, 9]]]
[[[[[4], [2], 5, [3, 5, 9], 6], 6], [7, [6, 4]], [[[8, 4, 0, 2, 8]], 0, 4

## Helper Functions

### Integer Comparator
3-way compare between two integers.

In [7]:
%%ipytest -xrPvvvvv

def compare_ints(num1: int, num2: int) -> int:
    if num1 > num2:
        return 1
    if num1 < num2:
        return -1
    return 0

def test_helper_1() -> None:
    assert compare_ints(0, 1) == -1
    assert compare_ints(1, 0) == 1
    assert compare_ints(1, 1) == 0

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


### Recursive List Comparator
If both arguments are integers, then the invariant base case of comparing two integers takes place. Otherwise, there is exactly 1 list and 1 integer, and we convert that integer to a list. Then, we perform a 3-way compare on the resultant lists.

The total ordering being asked of in the question is known as "lexicographical ordering" whereby the ordering between $a$ and $b$ is defined by the first index for which they differ. For example, "AAB" comes after "AAAA" because the first index for which they differ is the third letter, in which "B" > "A". Therefore, "AAB" > "AAAA". This is different from _shortlex order_ which is used in combinatorics where shorter items always come first (in this case, "AAB" < "AAAA")

Using the same 3-way comparator convention used in C, we return the comparison of $a$ and $b$ as follows:
- $-1$ if $a \lt b$
- $0$ if $a = b$
- $1$ if $a \gt b$

In [8]:
%%ipytest -xrPvvvvv

def compare_lists(first: list[Any] | int, second: list[Any] | int) -> int:

    t1 = type(first)
    t2 = type(second)

    if t1 == int and t2 == int:
        return compare_ints(first, second)

    if t1 == int:
        first = [first]

    if t2 == int:
        second = [second]

    for i in range(max(len(first), len(second))):
        if i >= len(first):
            return -1
        if i >= len(second):
            return 1
        if (val := compare_lists(first[i], second[i])) != 0:
            return val

    return 0

def test_helper_2() -> None:
    assert compare_lists([7, 7, 7, 7], [7, 7, 7]) == 1
    assert compare_lists([2, 3, 4], [4]) == -1
    assert compare_lists([1,1,3,1,1], [1,1,5,1,1]) == -1
    assert compare_lists([1,[2,[3,[4,[5,6,7]]]],8,9], [1,[2,[3,[4,[5,6,0]]]],8,9]) == 1
    assert compare_lists([[1],[2,3,4]], [[1],4]) == -1
    assert compare_lists([[[]]], [[]]) == 1

.                                                                                            [100%]
============================================= PASSES ==============================================
1 passed in 0.01s


## Main Function
Part 1: accumulate all pairs $a$ and $b$'s indices for which $a \lt b$ in the lexicographic ordering we defined assuming a 1-indexed list.
Part 2: sort the entire list using `cmp_to_key` and the 3-way comparator, then return the product of the two sentinel's indices, also 1-indexed. Fortunately the format I used to represent the comparator's outputs is is exactly the format Python (and many other languages) takes.

In [9]:
def solve(part: int, filename: str) -> int:
    input = parse_input_from_filename(filename).input
    if part == 1:
        ans = 0
        for idx, pair in enumerate(input):
            if compare_lists(pair[0], pair[1]) == -1:
                ans += idx + 1
        return ans
    if part == 2:
        all_lines = list(itertools.chain.from_iterable(input))
        all_lines.append([[2]])
        all_lines.append([[6]])
        all_lines.sort(key=cmp_to_key(lambda x, y: compare_lists(x, y)))
        return (1 + all_lines.index([[2]])) * (1 + all_lines.index([[6]]))
    else:
        raise Exception(f"Invalid part: {part}")

## Execution

### Part 1

In [10]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["example"].items())
def test_part_one_examples(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["input"].items())
def test_part_one_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

..                                                                                           [100%]
============================================= PASSES ==============================================
2 passed in 0.02s


### Part 2

In [11]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["example"].items())
def test_part_two_examples(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["input"].items())
def test_part_two_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

..                                                                                           [100%]
============================================= PASSES ==============================================
2 passed in 0.03s


Definitely could've done this one a little faster.